Hi,

This is a draft code that loads the pretrained model and predicts a given input. 
The notebook should run in less than five minutes.

Keep in mind that you should download the following folder (my saved model) and put it in the same folder as your code.

https://mega.nz/folder/MmB1gIIT#8ilUTK1-BO80aoXxKOIhpg

-Moradnejad

# Load the model

In [3]:
!pip install keras
!pip install tensorflow
import keras

model = keras.models.load_model("colbert-trained/")
model.summary()

     |████████████████████████████████| 511.7 MB 2.5 kB/s                           | 164.3 MB 29.2 MB/s eta 0:00:12 | 289.0 MB 24.1 MB/s eta 0:00:10
     |████████████████████████████████| 5.8 MB 32.3 MB/s            
     |████████████████████████████████| 4.5 MB 30.7 MB/s            
     |████████████████████████████████| 2.4 MB 16.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 438 kB 19.4 MB/s            
     |████████████████████████████████| 123 kB 37.5 MB/s            
     |████████████████████████████████| 77 kB 8.7 MB/s             
     |████████████████████████████████| 65 kB 10.4 MB/s            
     |████████████████████████████████| 57 kB 13.2 MB/s            
     |████████████████████████████████| 42 kB 3.2 MB/s             
     |████████████████████████████████| 14.1 MB 17.0 MB/s            
     |████████████████████████████████| 781 kB 27.5 MB/s            
     |████████████████████████████████| 167 kB 32.2 M

2022-08-02 19:19:18.586027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 19:19:18.586088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2022-08-02 19:19:22.099454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-02 19:19:22.099507: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-02 19:19:22.099535: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1966391d240c): /proc/driver/nvidia/version does not exist
2022-08-02 19:19:22.099741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

# complete code

In [4]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
cpu MHz		: 2400.000
cpu cores	: 1
# RAM
MemTotal:       28757784 kB
# GPU

# OS
Linux 1966391d240c 5.10.25-linuxkit #1 SMP Tue Mar 23 09:27:39 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


/bin/sh: 1: lspci: not found


In [6]:
!pip install transformers

import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

     |████████████████████████████████| 4.7 MB 5.5 MB/s            
     |████████████████████████████████| 101 kB 22.1 MB/s           
     |████████████████████████████████| 6.6 MB 13.2 MB/s            
     |████████████████████████████████| 765 kB 32.3 MB/s            
2.9.1


# Prep / tokenizer

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [14]:
training_sample_count = 1000 # 4000
test_count = 1000

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [8]:
!ls /kaggle/input/200k-short-texts-for-humor-detection

ls: cannot access '/kaggle/input/200k-short-texts-for-humor-detection': No such file or directory


### original dataset

In [18]:
# df = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')
df = pd.read_csv('Data/dataset.csv')

# df_train = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/train.csv')
df_train = pd.read_csv('Data/train.csv')
display(df_train.head(3))
df_train = df_train[:training_sample_count]

# df_test = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dev.csv')
df_test = pd.read_csv('Data/dev.csv')
display(df_test.head(3))
df_test = df_test[:test_count]

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True


,text,humor
0,What kind of cat should you take into the des...,True
1,Remember when people used to have to be in sha...,True
2,Pizza is always good. - everyone we'll see abo...,True


In [19]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

200000 1000 1000


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


,text
0,What kind of cat should you take into the des...
1,Remember when people used to have to be in sha...
2,Pizza is always good. - everyone we'll see abo...
3,"What's 6 inches long hard, bent, and in my pan..."
4,Black teen's response to violence in his commu...


In [20]:
print(list(df_train.columns))

['text', 'humor']


In [21]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['text']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


## 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.<br>


In [22]:
from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/jovyan/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f

In [23]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input



In [25]:
inputs      = compute_input_arrays(df_train, input_categories, tokenizer)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(1000, 20)


0it [00:00, ?it/s]

(1000, 20)


In [26]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))

# check out input for 7th row
xx = 7
print(df_train.iloc[xx,0])
print(sent_tokenize(df_train.iloc[xx,0]))
inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]

18 1000 20
Why do native americans hate it when it rains in april? because it brings mayflowers.
['Why do native americans hate it when it rains in april?', 'because it brings mayflowers.']


(array([  101,  2339,  2079,  3128,  4841,  5223,  2009,  2043,  2009,
        15811,  1999,  2258,  1029,   102,     0,     0,     0,     0,
            0,     0], dtype=int32),
 array([  101,  2138,  2009,  7545,  2089, 14156,  2015,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0], dtype=int32),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([  101,  2339,  2079,  3128,  4841,  5223,  2009,  2043,  2009,
        15811,  1999,  2258,  1029,  2138,  2009,  7545,  2089, 14156,
         2015,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,

In [27]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)
outputs[:3]

array([[False],
       [False],
       [ True]])

## 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 3 epochs --- with a learning rate of 3e-5 and batch_size of 6. A simple binary crossentropy is used as the objective-/loss-function. 

In [28]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

### Loss function selection
Regression problem between 0 and 1, so binary_crossentropy and mean_absolute_error seem good.

Here are the explanations: https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

In [29]:
valid_inputs = inputs
valid_outputs = outputs

In [30]:
preds = model.predict(valid_inputs)


32/32 [==============================] - 126s 4s/step


In [31]:
len(valid_inputs[0])

1000

In [32]:
print(valid_outputs.shape, preds.shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(preds), '')

(1000, 1) (1000, 1)
mean_absolute_error  : 0.008370925
mean_squared_error  : 0.005929642
r2 score  : 0.9762767811032094


0.005929642

In [33]:
test_preds = model.predict(test_inputs)

32/32 [==============================] - 136s 4s/step


## Binary submission

In [34]:
for split in np.arange(0.1, 0.99, 0.1).tolist():
    df_sub['pred_bi'] = (test_preds > split)

    print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

    df_sub.to_csv('sub3.csv', index=False)
    df_sub.head()

================== SPLIT on 0.1
f1_score  : 0.9769673704414588
[[467  17]
 [  7 509]]
Acc 0.976 Prec 0.967680608365019 Rec 0.9864341085271318 F1 0.9769673704414588
================== SPLIT on 0.2
f1_score  : 0.9797492767598842
[[471  13]
 [  8 508]]
Acc 0.979 Prec 0.9750479846449136 Rec 0.9844961240310077 F1 0.9797492767598842
================== SPLIT on 0.30000000000000004
f1_score  : 0.9816069699903194
[[474  10]
 [  9 507]]
Acc 0.981 Prec 0.9806576402321083 Rec 0.9825581395348837 F1 0.9816069699903194
================== SPLIT on 0.4
f1_score  : 0.9825581395348837
[[475   9]
 [  9 507]]
Acc 0.982 Prec 0.9825581395348837 Rec 0.9825581395348837 F1 0.9825581395348837
================== SPLIT on 0.5
f1_score  : 0.9835111542192047
[[476   8]
 [  9 507]]
Acc 0.983 Prec 0.9844660194174757 Rec 0.9825581395348837 F1 0.9835111542192047
================== SPLIT on 0.6
f1_score  : 0.9825242718446603
[[476   8]
 [ 10 506]]
Acc 0.982 Prec 0.9844357976653697 Rec 0.9806201550387597 F1 0.982524271844

In [35]:
df_sub['pred_bi'] = (test_preds > 0.5)

print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

df_sub.to_csv('sub.csv', index=False)
df_sub.head()

================== SPLIT on 0.9
f1_score  : 0.9835111542192047
[[476   8]
 [  9 507]]
Acc 0.983 Prec 0.9844660194174757 Rec 0.9825581395348837 F1 0.9835111542192047


,text,humor,pred_bi
0,What kind of cat should you take into the des...,True,True
1,Remember when people used to have to be in sha...,True,True
2,Pizza is always good. - everyone we'll see abo...,True,True
3,"What's 6 inches long hard, bent, and in my pan...",True,True
4,Black teen's response to violence in his commu...,False,False


In [36]:
print('Texts that the model failed to correctly predict:')
df_sub[df_sub['pred_bi']!=df_sub['humor']]

Texts that the model failed to correctly predict:


,text,humor,pred_bi
252,Ariana grande looks like she was designed in a...,True,False
303,"If you're happy and you know it, share your meds",True,False
309,Friends don't let friends make harlem shake' v...,True,False
310,The whip was especially popular in the 1800's,True,False
345,One out of five dentists has the courage to sp...,True,False
361,New subway slogan idea from jared 12 is the ne...,True,False
372,The easiest way to water plants is...using a t...,False,True
383,What is this 'wrong hole' you people speak of?,True,False
525,What happens when you retweet a compliment abo...,True,False
572,The boomerang is australia's chief export (and...,True,False
